# PRV Assignment 1

- Pablo Garcia Fernández

Initial configuration

In [41]:
# Initial configuration
import numpy as np
from math import cos,sin

#######################################
## PREDIFINE CONSTATNS OF THE EXERCISE

ROLL=2.5; PITCH=5; YAW=60 
TRANSLATION = np.array([0, 0, 0.25])

F = 2406.1 # focal length
CX = 25.498; CY=-33.0153 # principal point coordinates
K1 = 0.00247448; K2 = -0.0240244; K3 = 0.0604812; K4 = -0.0275491 # radial distorsion
P1 = 0.000150734; P2 = -0.000271032 # tangential distorsion
B1 = -5.67637; B2 = 0.339356 # affine

PIXEL_SIZE = 0.00156  # (mm)
W = 4000   # (px)
H = 3000   # (pX)

######################################

Auxiliar functions

In [42]:
def enu_to_ned_ned_to_enu():
    transformation_matrix = np.array([
        [0., 1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, -1, 0],
		[0, 0, 0, 1]
    ])
    return transformation_matrix


def _3D_rotation_matrix(roll, pitch, yaw, order='ypr', inverso=False):
	# Return the rotation matrix in 3D coordinate systems
	# Assumming ISPRS moving axes
	yaw = np.array([
		[cos(yaw), sin(yaw), 0, 0],
		[-sin(yaw), cos(yaw), 0, 0],
		[0, 0, 1, 0],
		[0, 0, 0, 1]
	])
	pitch = np.array([
		[cos(pitch), 0, -sin(pitch), 0],
		[0, 1, 0, 0],
		[sin(pitch), 0, cos(pitch), 0],
		[0, 0, 0, 1]
	])
	roll = np.array([
		[1, 0, 0, 0],
		[0, cos(roll), sin(roll), 0],
		[0, -sin(roll), cos(roll), 0],
		[0, 0, 0, 1]
	]) 

	rotation_matrix = None
	
	if order=='ypr':
		if inverso:
			rotation_matrix = yaw.T @ pitch.T @ roll.T
		else:
			rotation_matrix = roll @ pitch @ yaw
	elif order == 'rpy':
		if inverso:
			rotation_matrix = roll.T @ pitch.T @ yaw.T
		else:
			rotation_matrix = yaw @ pitch @ roll
	else:
		print("Order not supported")

	return rotation_matrix

### World CS -> INS CS
### INS CS -> INS CS

In [43]:
def ins_to_wcs():
    # Note -> WCS is ENU, INS is NED
        
    # 1) yaw, pitch, roll rotation
    t1 = _3D_rotation_matrix(ROLL, PITCH, YAW, inverso=True)
    
    # 2) NED to ENU
    t2 = enu_to_ned_ned_to_enu()
    
    return t2 @ t1

def wcs_to_ins():
    # 1) ENU to NED
    t1 = enu_to_ned_ned_to_enu()

    # 2) yaw, pitch, roll rotation
    t2 = _3D_rotation_matrix(ROLL, PITCH, YAW, inverso=False)

    return t2 @ t1

### INS CS -> Gimbal CS
### Gimbal CS -> INS CS 

In [44]:
# They are the same enu->ned/ned->enu and translation:

def gimbal_to_ins(traslation_vector):
    # 1) Translation + 2) ENU to NED
    transformation_matrix = enu_to_ned_ned_to_enu()
    transformation_matrix[:-1,-1] = traslation_vector

    return transformation_matrix

def ins_to_gimbal(traslation_vector):
    # 1) NED to ENU + 2) - translation
    transformation_matrix = enu_to_ned_ned_to_enu()
    transformation_matrix[:-1,-1] = traslation_vector

    return transformation_matrix


### Gimbal CS -> Camera CS
### Camera CS -> Gimbal CS

In [45]:
def camera_to_gimbal():
	# Frontal to nadiral
	# Rotation -90º axis X. -> X becomes X | Y becomes -Z | Z becomes Y
	t = np.array([
		[1, 0, 0, 0],
		[0, 0, -1, 0],
		[0, 1, 0, 0],
		[0, 0, 0, 1]
	]) 
	return t

def gimbal_to_camera():
	# Nadiral to frontal
	# Rotation 90º axis X -> X becomes X | Y becomes Z | Z becomes -Y
	t = np.array([
		[1, 0, 0, 0],
		[0, 0, 1, 0],
		[0, -1, 0, 0],
		[0, 0, 0, 1]
	]) 
	return t

### Composition of all transformations

In [46]:
# 1st path: camera to wcs
t1 = camera_to_gimbal()
print("camera to gimbal:\n", t1)
t2 = gimbal_to_ins(TRANSLATION)
print("gimbal to ins:\n", t2)
t3 = ins_to_wcs()
print("ins to wcs:\n",t3)
# TOTAL
t_c_w = t3 @ t2 @ t1
print("full matrix t_c_w:\n", t_c_w)
### Example of projection
punto = np.array([3, 4, 2, 1])
proyeccion = t_c_w @ punto
print("Point ", punto, " becomes ", proyeccion)
print("\n\n")

# 2nd path: wcs to camera
t1 = wcs_to_ins()
print("wcs to ins:\n", t1)
t2 = ins_to_gimbal(TRANSLATION)
print("ins to gimbal:\n", t2)
t3 = gimbal_to_camera()
print("gimbal to camera:\n",t3)
# TOTAL
t_w_c = t3 @ t2 @ t1
print("full matrix t_w_c:\n", t_w_c)
### Example of projection with the previous proyected point 'proyeccion'
punto = np.array([1.72742007, 4.87534516, 0.55635374, 1.])
proyeccion = t_w_c @ punto
print("Point ", punto, " becomes ", proyeccion)
print("\n\n")


# IMPORTANT!
# As expected the wcs-camera projection of the camera-wcs projected point is the same.
# [3, 4, 2, 1] -> [1.72742007, 4.87534516, 0.55635374, 1.] -> [3, 4, 2, 1]

### Another question:
# We can obtain the matrix of one path as the inverse of the other. 
# This means that the inverse of t_c_w is t_w_c. Proof 1:
print("The inverse of t_c_w must be t_w_c" )
print("Are the same? ->", np.allclose(np.linalg.inv(t_c_w), t_w_c))
print("t_w_c:\n", np.linalg.inv(t_c_w))

# Second proof t_c_w @ t_w_c must be I
print("2nd proof t_c_w @ t_w_c must be I")
print(t_c_w @ t_w_c)



camera to gimbal:
 [[ 1  0  0  0]
 [ 0  0 -1  0]
 [ 0  1  0  0]
 [ 0  0  0  1]]
gimbal to ins:
 [[ 0.    1.    0.    0.  ]
 [ 1.    0.    0.    0.  ]
 [ 0.    0.   -1.    0.25]
 [ 0.    0.    0.    1.  ]]
ins to wcs:
 [[-0.08646325  0.93794718  0.33582613  0.        ]
 [-0.27016355  0.30238269 -0.91409866  0.        ]
 [-0.95892427 -0.16976392  0.22725415  0.        ]
 [ 0.          0.          0.          1.        ]]
full matrix t_c_w:
 [[ 0.93794718 -0.33582613  0.08646325  0.08395653]
 [ 0.30238269  0.91409866  0.27016355 -0.22852467]
 [-0.16976392 -0.22725415  0.95892427  0.05681354]
 [ 0.          0.          0.          1.        ]]
Point  [3 4 2 1]  becomes  [1.72742007 4.87534516 0.55635374 1.        ]



wcs to ins:
 [[-0.08646325 -0.27016355 -0.95892427  0.        ]
 [ 0.93794718  0.30238269 -0.16976392  0.        ]
 [ 0.33582613 -0.91409866  0.22725415  0.        ]
 [ 0.          0.          0.          1.        ]]
ins to gimbal:
 [[ 0.    1.    0.    0.  ]
 [ 1.    0.    

## FRAME CAMERA MODEL

In [47]:
# CAMERA CS (3D x,y,z) -> IMAGE CS (2D u,v)
def camera_model(X,Y,Z):
    
    x = X/Z
    y = Y/Z
    r = np.sqrt(x**2 + y**2)
    
    x_prima = x*(1 + K1*r**2 + K2*r**4 + K3*r**6 + K4*r**8) + (P1*(r**2+2*x**2) + 2*P2*x*y)
    y_prima = y*(1 + K1*r**2 + K2*r**4 + K3*r**6 + K4*r**8) + (P2*(r**2+2*y**2) + 2*P1*x*y)
    
    u = W * 0.5 + CX + x_prima*F + x_prima*B1 + y_prima*B2
    v = H * 0.5 + CY + y_prima*F

    return u,v


# Test 
aa = camera_model(-0.7145984059926852, -0.5232285827705769, 1)
print(aa)

(299.99212544991826, 199.9938506827259)


## Camera calibration-undistort

<img src="asd.jpeg" width="800">

In [48]:
# IMAGE -> CAMERA
def image_to_camera(u,v, threshold, alpha = 0.00000001):
    
    # Obtain estimation pixels
    y_prima = (v - 0.5*H - CY) / F  
    x_prima = (u - 0.5*W - CX - y_prima*B2) / (F + B1) 

    # Initialize estimated undistorted coordinates and error
    x = x_prima
    y = y_prima
    error = np.Inf
    
    while(error > threshold):
        r = np.sqrt(x**2 + y**2)
        
        tangential = (1 + K1*r**2 + K2*r**4 + K3*r**6 + K4*r**8)
        u_prima, v_prima = camera_model(x,y,1)

        dl_uprima = (u_prima - u) / np.sqrt((u - u_prima)**2 + (v - v_prima)**2)
        dl_vprima = (v_prima - v) / np.sqrt((u - u_prima)**2 + (v - v_prima)**2)
       
        duprima_x = F * (tangential + 4*P1*x + 2*P2*y) + B1 * (tangential + 4*P1*x + 2*P2*y) + B2*2*P1*y
        dvprima_x = F*2*P1*y
        duprima_y = B2 * (tangential + 4*P2*y + 2*P1*x) + F*2*P2*x + B1*2*P2*x
        dvprima_y = F * (tangential + 4*P2*y + 2*P1*x)

        x = x - alpha * (dl_uprima*duprima_x + dl_vprima*dvprima_x)
        y = y - alpha * (dl_uprima*duprima_y + dl_vprima*dvprima_y)
        
        u_prima, v_prima = camera_model(x,y,1)
        error = np.sqrt((u - u_prima)**2 + (v - v_prima)**2)
        print(f"x: {x}, y: {y}")
        print(error)

# In some cases must be necessary tune alpha to converge
image_to_camera(300, 200, 0.01, alpha=0.00000001)


x: -0.7187373580463088, y: -0.5265567354745413
13.32196734747047
x: -0.7187185418339966, y: -0.5265415497461446
13.26132906325862
x: -0.7186997255212422, y: -0.5265263641820449
13.200691144467937
x: -0.7186809091074711, y: -0.5265111787829568
13.140053591104138
x: -0.7186620925921037, y: -0.5264959935496022
13.07941640316781
x: -0.7186432759745549, y: -0.5264808084827093
13.018779580664905
x: -0.7186244592542341, y: -0.5264656235830134
12.958143123597239
x: -0.7186056424305451, y: -0.5264504388512569
12.89750703196846
x: -0.7185868255028861, y: -0.526435254288189
12.836871305784044
x: -0.7185680084706494, y: -0.5264200698945667
12.776235945046082
x: -0.7185491913332216, y: -0.5264048856711537
12.715600949759315
x: -0.7185303740899832, y: -0.5263897016187216
12.654966319928555
x: -0.7185115567403089, y: -0.5263745177380491
12.594332055555387
x: -0.718492739283567, y: -0.5263593340299229
12.533698156645674
x: -0.7184739217191201, y: -0.5263441504951372
12.47306462320342
x: -0.71845510404